## Load *.trc files

Choose/insert a directory for the cell files to be searched in (e.g. F:\Marburg\single_colour_tracking\resting). All *.trc files in the directory will be loaded for the analysis.

In [1]:
from pySPT.widgets import widgetLoadHdf5
from pySPT.widgets import loadHdf5
from pySPT.analysis import cell
from pySPT.analysis import trajectory
from pySPT.analysis import coverSlip
from pySPT.analysis import trajectoryStatistics
widget_load_hdf5 = widgetLoadHdf5.WidgetLoadHdf5()
load_hdf5 = loadHdf5.LoadHdf5()
cover_slip = coverSlip.CoverSlip()
display(widget_load_hdf5.dir_box, widget_load_hdf5.dir_button)
widget_load_hdf5.dir_button.on_click(widget_load_hdf5.open_dir)
widget_load_hdf5.dir_box.observe(widget_load_hdf5.change_dir_box)

Text(value='directory to be searched in', description='directory', placeholder='Type something', style=Descrip…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [2]:
from pySPT.widgets.widgetNotebooks import init_filter_notebook
def initialize(event):
    init_filter_notebook(cover_slip, widget_load_hdf5, load_hdf5)
display(widget_load_hdf5.init_button)
widget_load_hdf5.init_button.on_click(initialize)

Button(description='initialize', style=ButtonStyle(), tooltip='initialize objects')

Initialization took 2.5442514419555664 s


## Filter for parameters

Floating point numbers have to be separated with a dot (e.g. 2.123). If no filter parameters or invalid parameters are given, the min and max values will be set automatically. All boxes that are checked will be included in the filtered data. If one wants to filter multiple times, the JNB has to be restarted.

In [3]:
track_stats = trajectoryStatistics.TrajectoryStatistics()
def filter(event):
    widget_load_hdf5.create_clear_output()
    track_stats.cell_trajectories = cover_slip.cell_trajectories
    track_stats.run_statistics(widget_load_hdf5.min_length_box.value, widget_load_hdf5.max_length_box.value, widget_load_hdf5.min_D_box.value, widget_load_hdf5.max_D_box.value, widget_load_hdf5.immob_type_check_box.value,  widget_load_hdf5.confined_type_check_box.value, widget_load_hdf5.free_type_check_box.value, widget_load_hdf5.analyse_successful_check_box.value, widget_load_hdf5.analyse_not_successful_check_box.value)
display(widget_load_hdf5.min_length_box, widget_load_hdf5.max_length_box, widget_load_hdf5.min_D_box, widget_load_hdf5.max_D_box, widget_load_hdf5.immob_type_check_box, widget_load_hdf5.confined_type_check_box, widget_load_hdf5.free_type_check_box, widget_load_hdf5.analyse_successful_check_box, widget_load_hdf5.analyse_not_successful_check_box, widget_load_hdf5.filter_button)
widget_load_hdf5.filter_button.on_click(filter)

max trajectory length: 538
min trajectory length: 8
max diffusion coefficient: 2.4615554891102156 [μm²/s]
min diffusion coefficient: 1e-05 [μm²/s]
15.8 % are immobile
38.1 % are confined
46.1 % are free


In [4]:
# trajectory_stats.cell_trajectories = cover_slip.cell_trajectories
# trajectory_stats.get_index()
# trajectory_stats.create_init_filter_lst()

## Choose trajectory to plot

Choose a cell. If one wants to plot multiple times, choose a new cell and/or trajectory reload the plotting cell by clicking in the cell and pressing strg+enter.

In [5]:
def dropdown(event):
    widget_load_hdf5.cell_options = widget_load_hdf5.get_cell_names(cover_slip.cells)
widget_load_hdf5.filter_button.on_click(dropdown)
#widget_load_hdf5.filter_button.observe(dropdown)
display(widget_load_hdf5.drop_down_cells)

Dropdown(description='Number:', options=(), value=None)

Choose a trajectory.

In [6]:
def dropdown(event):      
    for cell in cover_slip.cells:
        if cell.name == widget_load_hdf5.drop_down_cells.value:
            target_cell = cover_slip.cells.index(cell)
            widget_load_hdf5.trajectory_options = track_stats.cell_trajectories_filtered_index[target_cell]
            widget_load_hdf5.drop_down_trajectories.options = widget_load_hdf5.trajectory_options
widget_load_hdf5.drop_down_cells.observe(dropdown)
widget_load_hdf5.filter_button.on_click(dropdown)
display(widget_load_hdf5.drop_down_trajectories)

Dropdown(description='Number:', options=(), value=None)

In [7]:
def plot_trajectory(event):
    widget_load_hdf5.create_clear_output()
    cover_slip.plot_trajectory(widget_load_hdf5.drop_down_cells.value, widget_load_hdf5.drop_down_trajectories.value)
display(widget_load_hdf5.plot_button)
widget_load_hdf5.plot_button.on_click(plot_trajectory)

Button(description='plot', style=ButtonStyle(), tooltip='plot chosen trajectory')

## Plot diffusion histogram

In [8]:
def plot_diffusions(event):
    print("xxx")
    track_stats.background_frequencies_lst()
display(widget_load_hdf5.plot_diffusions_button)
widget_load_hdf5.plot_diffusions_button.on_click(plot_diffusions)

Button(description='plot', style=ButtonStyle(), tooltip='plot diffusion coefficients')

xxx


[[-1.5210343096611039, -1.8086842584234044, -2.4919877364603855, -0.7859672044840251, -0.8908288785407653, -0.8404260399443296, -2.9793583583991983, -0.9524734667680425, -0.8084773712595112, -1.1890715617181604, -1.3343490947481247, -0.7939547845648843, -1.5532712965841948, -0.9424217712592882, -0.5985161020018073, -1.366805053788057, -1.0665844293663413, -0.6220965317071416, -1.6230106150560992, -0.7353044424268409, -2.5527441436850387, -1.5513854981750534, -0.18314944651786874, -0.5208906203262594, -2.189992107532771, -1.4244145091687184, -1.6026927785331546, -0.6933969383733392, -0.7581856968993304, -1.0779462118020602, -0.6257135450792147, -1.7527570208244465, -1.1774408066267292, -5.0, -1.7444216017345113, -0.4910308101765368, -2.683015122067916, -0.9336933790501186, -1.2097387072177928, -1.6406619480194191, -0.976442387086941, -0.759645881378605, -1.0146569200931042, -2.091677061278338, -1.0300413307116625, -1.0523801223021585, -0.7858545892849915, -1.3503217157642173, -1.4019225